In [1]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트


In [2]:
# 패키지 설치

import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from glob import glob
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import nltk
import joblib
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 메타 전문가 리뷰

critic_data = pd.read_csv('./data/meta_final/metacrawl_meta_final.csv')

# game / review 열로 가져오기, 조건 설정
d1 = critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)][['title', 'criticcontent']]

# 이름 각각 game, score으로 변경
d1.rename(columns = {'title':'game', 'criticcontent':'review'}, inplace = True)

d1.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
def remain_engnum(sentence):
  sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence)
  return sentence

tqdm.pandas()
d1['game'] = d1['game'].progress_apply(lambda x:remain_engnum(x))
d1['game'] = d1['game'].str.replace(" ", "")
d1['game'] = d1['game'].str.lower()


# 메타 유저 리뷰

user_data = pd.read_csv("./data/meta_final/metacrawl_user_final.csv")

# game / review 열로 가져오기, 조건 설정
d2 = user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)][['title', 'usercontent']]

# 이름 각각 user, game, score으로 변경
d2.rename(columns = {'title':'game', 'usercontent':'review'}, inplace = True)

d2.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
d2['game'] = d2['game'].progress_apply(lambda x:remain_engnum(x))
d2['game'] = d2['game'].str.replace(" ", "")
d2['game'] = d2['game'].str.lower()

# 스팀 리뷰 데이터

steam_review = pd.read_csv("./data/steam_data/real_final.csv")

steam_review.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거
steam_review.dropna(subset=['weighted_vote_score'], inplace=True) # weighted_vote_score NaN값인 행 제거
steam_review.dropna(subset=['voted_up'], inplace=True) # voted_up NaN값인 행 제거
steam_review.dropna(subset=['title'], inplace=True) # title NaN값인 행 제거

d3 = steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)][['title', 'review']]

# 이름 각각 game, score으로 변경
d3.rename(columns = {'title':'game'}, inplace = True)

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
d3['game'] = d3['game'].progress_apply(lambda x:remain_engnum(x))
d3['game'] = d3['game'].str.replace(" ", "")
d3['game'] = d3['game'].str.lower()


# d1, d2, d3 합치기
total_data = pd.concat([d1, d2, d3])
# nan값 있는 행 제거
total_data.dropna(inplace=True)
# 게임 명 공백인 행 제거
drop_index = total_data[total_data['game']==''].index
total_data.drop(drop_index, axis='index', inplace=True)

total_data.head()

100%|██████████| 212595/212595 [00:00<00:00, 311570.86it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 46875/46875 [00:00<00:00, 284613.27it/s]


,game,review
0,discoelysiumthefinalcut,Traversing the mind of your character is as mu...
1,discoelysiumthefinalcut,Disco Elysium – The Final Cut is something rar...
2,discoelysiumthefinalcut,"Enhanced with fantastic and fresh content, Dis..."
3,discoelysiumthefinalcut,A fully voiced cast and new content of compara...
4,discoelysiumthefinalcut,Disco Elysium: The Final Cut doesn't shy away ...


In [ ]:
# 희귀단어 제거 토크나이저 만들기

# tokenizer로 total_data['review'] 단어사전 만들고 희귀단어 비율 보기

tok = Tokenizer()
tok.fit_on_texts(total_data['review'])

# 등장 빈도수가 3회 이하 단어들이 이 데이터에서 얼마만큼 비중을 차지 하는지 확인

threshold = 3
total_cnt = len(tok.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

for key, value in tok.word_counts.items():
    total_freq = total_freq + value
    if(value <= threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 희귀단어 빼고 정수인덱싱

vocab_size = total_cnt- rare_cnt + 1 # 패딩사이즈 고려해서 단어사전 크기 설정
print("단어 사전 크기 :", vocab_size)

단어 집합(vocabulary)의 크기 : 279634
등장 빈도가 3번 이하인 희귀 단어의 수: 208338
단어 집합에서 희귀 단어의 비율: 74.50381570195327
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.0714558277484987
단어 사전 크기 : 71297


In [ ]:
# 전처리

# 영어, 숫자만 빼고 제거
total_data['review'] = total_data['review'].progress_apply(lambda x:remain_engnum(x))

# 소문자화
total_data['review']= total_data['review'].apply(lambda x: x.lower())

# 게임 별로 리뷰 합치기
game_review = pd.DataFrame(total_data['game'].unique(), columns=['game'])

for game in tqdm(total_data['game'].unique()):
  game_review.loc[game_review['game']==game, 'review'] = " ".join(total_data[total_data['game']==game]['review'])

# 희귀단어 제거를 위한 토크나이저 생성
vocab_size = 71297 # 단어사전 크기
total_tokenizer = Tokenizer(num_words = vocab_size) # 단어사전 생성
total_tokenizer.fit_on_texts(game_review['review']) # 학습

# 단어사전에서 인덱스가 71297 초과인 단어(=희귀단어) 제거
words_frequency = [word for word, index in total_tokenizer.word_index.items() if index >= vocab_size + 1]
for word in words_frequency:
    del total_tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del total_tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

# 토큰화
game_review['review'] = game_review['review'].progress_apply(lambda x:text_to_word_sequence(x))

# 희귀단어 제거 함수
def remain_word(review):
  result = []
  for word in review:
    if word in total_tokenizer.word_index.keys():
      result.append(word)
  return result

game_review['review'] = game_review['review'].progress_apply(lambda x:remain_word(x))

# 불용어 제거
stop_words = set(stopwords.words('english'))

def remove_stopwords(text): 
    output= [i for i in text if i not in stop_words] 
    return output

game_review['review'] = game_review['review'].progress_apply(lambda x:remove_stopwords(x))

100%|██████████| 13195/13195 [00:03<00:00, 4045.04it/s]


In [ ]:
# 데이터에 태그 달기

doc_df = game_review[['game','review']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[gid]) for gid, _d in doc_df]

max_epochs = 10

model = Doc2Vec(
    window=5, # 모델 학습 시 앞 뒤로 보는 단어 개수
    size=100, # 벡터차원 크기
    alpha=0.025, # learning rate
    min_alpha=0.025,# min learning rate. 학습 진행될수록 alpha값이 min_alpha로 근사
    min_count=3, # 학습에 사용할 최소 단어 빈도 수
    dm =1, # 학습 방법 : 1-pvdm, 0-pvdbow
    negative = 5, # complexity reudction. negative sampling 시 샘플링 할 단어 개수 지정
    seed = 9999)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save('./model/finaldatadoc2vec.model')

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [3]:
# 나중엔 여기부터

model = Doc2Vec.load('./model/finaldatadoc2vec.model')

In [ ]:
# mydoc2vecmodel3을 pickle로 저장
# joblib.dump(model, './doc2vec_proto.pkl')

# 로딩
# model_test = joblib.load('./doc2vec_proto.pkl')

['./doc2vec_proto.pkl']

In [ ]:
model.docvecs.most_similar(['stardewvalley'], topn=5)

[('graveyardkeeper', 0.6194978952407837),
 ('mytimeatportia', 0.6133836507797241),
 ('spiritfarer', 0.6030116081237793),
 ('moonlighter', 0.5977514982223511),
 ('catcafemanager', 0.5900871753692627)]

In [ ]:
model.docvecs.most_similar(['stardewvalley'], topn=5)

[('slimerancher', 0.6361432671546936),
 ('spiritfarer', 0.625251054763794),
 ('mytimeatportia', 0.6062695384025574),
 ('grow:songoftheevertree', 0.6025106906890869),
 ('ashorthike', 0.598648726940155)]

In [ ]:
model.docvecs.most_similar(negative = ['stardewvalley'], topn=5)

[('quake4', 0.30374395847320557),
 ('f.e.a.r.', 0.2796289622783661),
 ('unrealtournament2004', 0.26523882150650024),
 ('wolfenstein', 0.2598583400249481),
 ('callofduty', 0.25443947315216064)]

In [ ]:
model.docvecs.most_similar(negative = ['stardewvalley'], topn=5)

[('starwars:republiccommando', 0.3022231161594391),
 ('startrek:eliteforceii', 0.263746976852417),
 ('f.e.a.r.', 0.2607755959033966),
 ('aliens:colonialmarines', 0.24807630479335785),
 ('f.e.a.r.3', 0.23562407493591309)]

In [63]:
with open('./data/finaltagdata.pkl', 'rb') as f: # wb : 피클 작성 시 / rb : 피클 열람 시
  tagged_data = pickle.load(f)

In [66]:
tagged_data[0]

TaggedDocument(words=['traversing', 'mind', 'character', 'much', 'plot', 'case', 'trying', 'solve', 'immersed', 'world', 'revachol', 'struggling', 'leave', 'let', 'bugs', 'lack', 'podcasters', 'put', 'spending', 'time', 'delirious', 'oil', 'painted', 'tragicomedy', 'disco', 'elysium', 'sort', 'game', 'play', 'master', 'kind', 'game', 'truly', 'internalize', 'thoughts', 'make', 'real', 'life', 'easier', 'live', 'keep', 'company', 'wait', 'without', 'power', 'glorious', 'future', 'never', 'come', 'even', 'would', 'end', 'stupid', 'disco', 'elysium', 'final', 'cut', 'something', 'rare', 'issued', 'version', 'already', 'beloved', 'game', 'manages', 'eclipse', 'source', 'material', 'clever', 'thoughtful', 'additions', 'maximise', 'one', 'game', 'beautiful', 'assets', 'prose', 'crpg', 'truly', 'claim', 'emulate', 'tabletop', 'experience', 'adherence', 'statistics', 'rules', 'character', 'sheets', 'simple', 'act', 'asking', 'questions', 'thoughtfully', 'incorporating', 'answers', 'enhanced', 

# 하이퍼파리미터 최적화

- GridsearchCV로 파라미터 바꾸기, knn으로 output간 유사도 계산

## GridsearchCV

In [ ]:
import sys
import os
from time import time
from operator import itemgetter
import pickle
import pandas as pd
import numpy as np
from argparse import ArgumentParser

from gensim.models.doc2vec import Doc2Vec
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from gensim.models import KeyedVectors
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

from sklearn.base import BaseEstimator
# from gensim import corpora

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


dataset = pd.read_csv("data.csv")

class Doc2VecModel(BaseEstimator):

    def __init__(self, dm=1, size=1, window=1):
        self.d2v_model = None
        self.size = size
        self.window = window
        self.dm = dm

    def fit(self, raw_documents, y=None):
        # Initialize model
        self.d2v_model = Doc2Vec(size=self.size, window=self.window, dm=self.dm, iter=5, alpha=0.025, min_alpha=0.001)
        # Tag docs
        tagged_documents = []
        for index, row in raw_documents.iteritems():
            tag = '{}_{}'.format("type", index)
            tokens = row.split()
            tagged_documents.append(TaggedDocument(words=tokens, tags=[tag]))
        # Build vocabulary
        self.d2v_model.build_vocab(tagged_documents)
        # Train model
        self.d2v_model.train(tagged_documents, total_examples=len(tagged_documents), epochs=self.d2v_model.iter)
        return self

    def transform(self, raw_documents):
        X = []
        for index, row in raw_documents.iteritems():
            X.append(self.d2v_model.infer_vector(row))
        X = pd.DataFrame(X, index=raw_documents.index)
        return X

    def fit_transform(self, raw_documents, y=None):
        self.fit(raw_documents)
        return self.transform(raw_documents)


param_grid = {'doc2vec__window': [2, 3],
              'doc2vec__dm': [0,1],
              'doc2vec__size': [100,200],
              'logreg__C': [0.1, 1],
}

pipe_log = Pipeline([('doc2vec', Doc2VecModel()), ('log', LogisticRegression())])

log_grid = GridSearchCV(pipe_log, 
                        param_grid=param_grid,
                        scoring="accuracy",
                        verbose=3,
                        n_jobs=1)

fitted = log_grid.fit(dataset["posts"], dataset["type"])

# Best parameters
print("Best Parameters: {}\n".format(log_grid.best_params_))
print("Best accuracy: {}\n".format(log_grid.best_score_))
# print("Finished.")

## KNN

In [ ]:
params = {
    'dm': (1, ), # 학습 방법 : 1-pvdm, 0-pvdbow
    'size': (100, 250, 500), # 벡터차원 크기
    'window': (3, 5, 8, 10, 15), # 모델 학습 시 앞 뒤로 보는 단어 개수
    'mean': (0, 1), # 0일 경우 단어 벡터의 합 이용, 1일 경우 평균 이용 / dm_concat이 0인 경우만 적용 가능
    'concat': (0, 1), # 1인 경우 문서 벡터로서 합 또는 평균 이용하지 않고 연속 연결로 사용, 훨씬 큰 모델 생성
    'min_count': (3, 5, 10, 30, 100), # 학습에 사용할 최소 단어 빈도 수
}

def run1():
    train_docs = read_corpus(data_dir, 0, train_percent)
    
    for dm in params['dm']:
        for size in params['size']:
            for window in params['window']:
                for mean in params['mean']:
                    for concat in params['concat']:
                        for min_count in params['min_count']:
                            doc2vec = Doc2Vec(dm=dm, size=size, window=window, dm_mean=mean, dm_concat=concat, min_count = min_count)
                            doc2vec.train(train_docs) # model의 neural weight 갱신
                            if doc2vec.from_file:
                                sys.stderr.write('!!!Ignore:' + doc2vec.auto_name() + '\n')
                                continue
                            knn = KNN(doc2vec)
                            acc = knn.score(train_docs)
                            print('!!RESULT!!====dm={},size={},window={},mean={},concat={},min_count={},acc={}'.format(dm, size, window, mean, concat,min_count, acc))
                            sys.stderr.write('!!RESULT!!====dm={},size={},window={},mean={},concat={},min_count={},acc={}\n'.format(dm, size, window, mean, concat,min_count, acc))
    print('DONE')

# 게임 input 3개 들어왔을 때 게임 추천 6개 뱉는 함수 생성
- 구현할 수는 있는데
- 서비스 페이지에서 장르 선택 탭 없애서 → input 게임끼리 추천 리스트가 서로 겹치지 않을 것
- 하지만 사실 장르 선택을 한다고 해도 겹칠 확률 거의 없어보임
  - stardewvalley의 추천 목록과 stardewvalley의 유사 게임 slimerancher의 추천 목록 중 겹치는 게임 수 : 2개
  - overwatch와 valorant의 추천 게임 리스트 중 겹치는 게임 수 : 5개
- 따라서 평균 유사도 높은 순으로 5개 추천하는 것 그닥 좋지 않아보임
- 아니다 좋을 수도
- 어차피 겹치는 게임을 추천해줘야하고 겹치면 평균치도 높아짐(0 개수 줄어드니까)
- 겹치는 게임 없는 경우 : 그냥 유사도 높은 순으로 추천해줌
  - 이게 문제네
  - 한 게임 추천 리스트에서만 유사도 높은 게임 많이 나오면 어떡함
  - 다른 input게임에 대한 추천리스트는 거의 반영되지 않게 됨
- input 게임 하나 당 추천 게임 2개씩 뱉으면 이런 일은 없음
- 그래서 어떡할까? → 물어보기
- 한 게임 당 2개씩 총 6개 띄우기로 함
- 근데 이건 추천 리스트 겹치면 어떡하지
- 20개 받아오고 중복 제거한 상태에서 5개 남기기
- **유사도 평균 내서 상위 6개 뽑는데**
- **중복게임 없는 경우 그냥 추천리스트에서 각 2개씩 뽑음**

In [59]:
# game1, game2, game3 추천 게임 - 유사도 테이블
def gentleman_ver1(game1, game2, game3):
  d1 = pd.DataFrame(model.docvecs.most_similar([game1], topn=10), columns=['game', 'similarity1']) # game1 추천목록 & 유사도
  d2 = pd.DataFrame(model.docvecs.most_similar([game2], topn=10), columns=['game', 'similarity2']) # game2 추천목록 & 유사도
  d3 = pd.DataFrame(model.docvecs.most_similar([game3], topn=10), columns=['game', 'similarity3']) # game3 추천목록 & 유사도
  df = pd.merge(d1, d2, on='game', how='outer') # d1, d2 merge
  df = pd.merge(df, d3, on='game', how='outer') # d2, d3 merge

  for game in df['game']: # ['game'] 행 하나씩 돌면서 
    if game in [game1, game2, game3]: # game이 game1, game2,m game3에 해당하는 경우
      drop_index = df[df['game']==game].index
      df.drop(drop_index, axis='index', inplace=True) # 해당 행 drop

  # 추천 리스트 간 중복되는 게임이 없는 경우 각 추천리스트에서 2개씩 뽑음
  if len(df) == 30: 
    return [x[0] for x in model.docvecs.most_similar([game1], topn=2)] + [x[0] for x in model.docvecs.most_similar([game2], topn=2)] + [x[0] for x in model.docvecs.most_similar([game3], topn=2)]
  # 유사도 평균 내서 상위 6개 추출
  else : 
    df.fillna(0, inplace=True)
    df['mean'] = df.mean(axis=1, numeric_only=True)
    df.sort_values('mean', ascending=False, inplace=True)
    return list(df['game'])[:6]

In [60]:
gentleman_ver1('overwatch', 'valorant', 'paladins')

['dirtybomb',
 'lawbreakers',
 'roguecompany',
 'heroesofthestorm',
 'battleborn',
 'apexlegends']

# 앱id - 게임이름 연결 table
- 호성님 요청 자료
  - 내 모델에 쓰이는 게임 리스트
  - 게임이름 전처리 안 된 버전을 옆 컬럼에 붙여서
  - 피클로

## 게임 이름 중복 유무 확인
- critic_data, user_data, steam_review 간 동명의 다른 게임 있는지 확인
  - 세 데이터 모두 조건 걸어서 행 추출 후 d1, d2, d3으로 변환했으므로
  - 동일한 조건으로 행 추출후 d1, d2, d3과 비교
- 결과
  - critic_data내에서는 동명의 다른 게임 없음
  - user_data에서 2개, steam_review에서 8개 발생했는데 뭔지는 찾아봐야함
  - d1, d2, d3 간 겹치는 게임 많음 : 이건 당연함
- 결론
  - 원래는 다른 게임인데
  - 게임 이름 전처리해서 같게 처리되는 게임이 있다
  - 근데 그냥 같은 걸로 보고 appid 같은 걸로 부여할 거임 수고해

In [ ]:
print(critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)]['title'].nunique(), d1['game'].nunique())

6199 6199


In [ ]:
print(user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)]['title'].nunique(), d2['game'].nunique())

9062 9060


In [ ]:
print(steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)]['title'].nunique(), d3['game'].nunique())

5174 5166


In [ ]:
print(d1['game'].nunique() + d2['game'].nunique() + d3['game'].nunique(), total_data['game'].nunique())

20425 13469


In [ ]:
print(sorted(total_data['game'].unique().tolist()))

['007legends', '007nightfire', '007quantumofsolace', '0n0w', '0rbitalis', '10000000', '1000amps', '1001spikes', '100hiddenbirds', '100hiddendogs', '100hiddeneternals', '100nya', '100orangejuice', '100orangejuicebreakerpack', '100orangejuicecharactersongpackultimateweapongirl', '100orangejuicechriskyupitacharacterpack', '100orangejuicecorevoicepack1', '100orangejuicekirikononamepack', '100orangejuicemixedboosterpack', '100orangejuicepuddingpack', '100orangejuicetsihtequilacharacterpack', '100orangejuicewandererpack', '100orangejuicewitchpack', '100orangejuiceyukitomomorenewalpack', '10secondninja', '10secondninjax', '1111memoriesretold', '112operator', '112operatormaskscharitydlc', '112operatorthelastduty', '123kickitdropthatbeatlikeanuglybaby', '123slaughtermestreet2', '12isbetterthan6', '12laboursofhercules', '12laboursofherculesiiigirlpower', '12laboursofherculesiithecretanbull', '12laboursofherculesivmothernature', '12laboursofherculesviifleecingthefleece', '12laboursofherculesvirac

## 호성님 요청 자료
- 내 모델에 쓰이는 게임 리스트
- 게임이름 전처리 안 된 버전을 옆 컬럼에 붙여서
- 피클로

In [ ]:
# 내 모델에 쓰이는 게임 리스트
gamelist = total_data['game'].unique().tolist()
gamelist

In [ ]:
with open('gamelist.pkl', 'wb') as f: # wb : 피클 작성 시 / rb : 피클 열람 시
	pickle.dump(gamelist, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(gamelist)

['discoelysiumthefinalcut', 'halflife2', 'grandtheftautov', 'outoftheparkbaseball2007', 'theorangebox', 'halflife', 'bioshock', 'baldursgateiishadowsofamn', 'divinityoriginalsiniidefinitiveedition', 'portal2', 'theelderscrollsvskyrim', 'commandconquer', 'masseffect2', 'eldenring', 'grandtheftautovicecity', 'sidmeierscivilizationii', 'quake', 'bioshockinfinite', 'theelderscrollsivoblivion', 'grimfandango', 'divinityoriginalsinenhancededition', 'diablo', 'sidmeierscivilizationiv', 'thewitcher3wildhunt', 'companyofheroes', 'tetriseffectconnected', 'halflifealyx', 'divinityoriginalsinii', 'unrealtournament2004', 'starcraftiiwingsofliberty', 'minecraft', 'reddeadredemption2', 'homeworld', 'grandtheftautoiii', 'godofwar', 'beatsaber', 'hades', 'starwarsknightsoftheoldrepublic', 'worldofwarcraft', 'grandtheftautosanandreas', 'callofduty4modernwarfare', 'warcraftiiireignofchaos', 'sidmeiersgettysburg', 'thesims', 'thesimsdeluxe', 'worldsoccerwinningeleven7international', 'teamfortress2', 'syst

In [ ]:
print(len(total_data['game']), len(gamelist))

321701 13195


gamelist 전처리 안된 버전 얻으려면 그냥 전처리 과정을 데이터 프레임 전체로 하면 되겠구나
- critic_data, user_data, steam_review는 그대로 사용
- df1, df2, df3으로 조건 설정 행 추출만 함
- 게임 이름 전처리를 새 열로
- concat까지 하고 '' 드롭
- 이 상태에서 for문 돌려서 unique한 것만 뽑아내면 될 것 같은데

In [ ]:
# 메타 전문가 리뷰

# game / review 열로 가져오기, 조건 설정
df1 = critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)][['title', 'criticcontent']]

# 열이름 각각 game, score으로 변경
df1.rename(columns = {'title':'game', 'criticcontent':'review'}, inplace = True)

df1.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거

df1['game_newname'] = df1['game'].progress_apply(lambda x:remain_engnum(x))
df1['game_newname'] = df1['game_newname'].str.replace(" ", "")
df1['game_newname'] = df1['game_newname'].str.lower()


# 메타 유저 리뷰

# game / review 열로 가져오기, 조건 설정
df2 = user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)][['title', 'usercontent']]

# 이름 각각 user, game, score으로 변경
df2.rename(columns = {'title':'game', 'usercontent':'review'}, inplace = True)

df2.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
df2['game_newname'] = df2['game'].progress_apply(lambda x:remain_engnum(x))
df2['game_newname'] = df2['game_newname'].str.replace(" ", "")
df2['game_newname'] = df2['game_newname'].str.lower()


# 스팀 리뷰 데이터

df3 = steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)][['title', 'review']]

# 이름 각각 game, score으로 변경
df3.rename(columns = {'title':'game'}, inplace = True)

# 게임 이름에 전처리 : 특수문자 없애기, 띄어쓰기 없애기, 소문자 처리하기
df3['game_newname'] = df3['game'].progress_apply(lambda x:remain_engnum(x))
df3['game_newname'] = df3['game_newname'].str.replace(" ", "")
df3['game_newname'] = df3['game_newname'].str.lower()


# df1, df2, df3 합치기
new_data = pd.concat([df1, df2, df3])
# nan값 있는 행 제거
new_data.dropna(inplace=True)
# 게임 명 공백인 행 제거
drop_index = new_data[new_data['game_newname']==''].index
new_data.drop(drop_index, axis='index', inplace=True)

new_data.head()

100%|██████████| 46875/46875 [00:00<00:00, 281848.60it/s]


,game,review,game_newname
0,Disco Elysium: The Final Cut,Traversing the mind of your character is as mu...,discoelysiumthefinalcut
1,Disco Elysium: The Final Cut,Disco Elysium – The Final Cut is something rar...,discoelysiumthefinalcut
2,Disco Elysium: The Final Cut,"Enhanced with fantastic and fresh content, Dis...",discoelysiumthefinalcut
3,Disco Elysium: The Final Cut,A fully voiced cast and new content of compara...,discoelysiumthefinalcut
4,Disco Elysium: The Final Cut,Disco Elysium: The Final Cut doesn't shy away ...,discoelysiumthefinalcut


In [ ]:
new_data.drop(['review'],axis=1, inplace=True)
new_data.head()

,game,game_newname
0,Disco Elysium: The Final Cut,discoelysiumthefinalcut
1,Disco Elysium: The Final Cut,discoelysiumthefinalcut
2,Disco Elysium: The Final Cut,discoelysiumthefinalcut
3,Disco Elysium: The Final Cut,discoelysiumthefinalcut
4,Disco Elysium: The Final Cut,discoelysiumthefinalcut


In [ ]:
print(new_data['game'].nunique())
print(new_data['game_newname'].nunique())

13676
13195


In [ ]:
new_data.drop_duplicates(['game_newname'], keep = 'first', inplace=True)

In [ ]:
with open('gamelistdf.pkl', 'wb') as f: # wb : 피클 작성 시 / rb : 피클 열람 시
	pickle.dump(new_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('./data/gamelistdf.pkl', 'rb') as f: # wb : 피클 작성 시 / rb : 피클 열람 시
  data = pickle.load(f)

In [7]:
data.head()

,game,game_newname
0,Disco Elysium: The Final Cut,discoelysiumthefinalcut
11,Half-Life 2,halflife2
92,Grand Theft Auto V,grandtheftautov
149,Out of the Park Baseball 2007,outoftheparkbaseball2007
154,The Orange Box,theorangebox
